![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_DeBertaForZeroShotClassification.ipynb)

## Import ONNX BertForZeroShotClassification  models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `BertForZeroShotClassification ` is only available since in `Spark NLP 5.2.4` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import Deberta models trained/fine-tuned for zero shot classification via `DebertaV2ForSequenceClassification` or `TFDebertaV2ForSequenceClassification`. These models are usually under `Zero-Shot Classification` category and have `bert` in their labels
- Reference: [TFDebertaV2ForSequenceClassification](https://huggingface.co/docs/transformers/en/model_doc/deberta-v2#transformers.TFDebertaV2ForSequenceClassification)
- Some [example models](https://huggingface.co/models?pipeline_tag=zero-shot-classification&sort=downloads&search=deberta)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.51.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [ ]:
!pip install -q transformers[onnx]==4.51.3 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli](https://huggingface.co/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli) model from HuggingFace as an example and load it as a `ORTModelForSequenceClassification`, representing an ONNX model.
- In addition to the DeBERTa model, we also need to save the tokenizer. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import DebertaV2Tokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification

MODEL_NAME = 'MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL)

tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(ONNX_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

('onnx_models/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli/tokenizer_config.json',
 'onnx_models/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli/special_tokens_map.json',
 'onnx_models/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli/spm.model',
 'onnx_models/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {ONNX_MODEL}

total 723692
-rw-r--r-- 1 root root        23 Jun 12 01:16 added_tokens.json
-rw-r--r-- 1 root root      1022 Jun 12 01:16 config.json
-rw-r--r-- 1 root root 738574335 Jun 12 01:16 model.onnx
-rw-r--r-- 1 root root       970 Jun 12 01:16 special_tokens_map.json
-rw-r--r-- 1 root root   2464616 Jun 12 01:16 spm.model
-rw-r--r-- 1 root root      1314 Jun 12 01:16 tokenizer_config.json


- As you can see, we need to move `vocabs.txt` from the tokenizer to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [4]:
!mkdir {ONNX_MODEL}/assets

labels = ort_model.config.id2label
labels = [value for key, value in sorted(labels.items())]

with open(f"{ONNX_MODEL}/assets/labels.txt", 'w') as f:
    f.write('\n'.join(labels))

!mv {ONNX_MODEL}/spm.model  {ONNX_MODEL}/assets

In [9]:
!ls -lR {ONNX_MODEL}

onnx_models/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli:
total 721288
-rw-r--r-- 1 root root        23 Jun 12 01:16 added_tokens.json
drwxr-xr-x 2 root root      4096 Jun 12 01:16 assets
-rw-r--r-- 1 root root      1022 Jun 12 01:16 config.json
-rw-r--r-- 1 root root 738574335 Jun 12 01:16 model.onnx
-rw-r--r-- 1 root root       970 Jun 12 01:16 special_tokens_map.json
-rw-r--r-- 1 root root      1314 Jun 12 01:16 tokenizer_config.json

onnx_models/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli/assets:
total 2412
-rw-r--r-- 1 root root      32 Jun 12 01:16 labels.txt
-rw-r--r-- 1 root root 2464616 Jun 12 01:16 spm.model


In [8]:
!cat {ONNX_MODEL}/assets/labels.txt

entailment
neutral
contradiction

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

## Import and Save DeBertaForZeroShotClassification in Spark NLP


- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

- **Optional: Use the latest versions**
  - If you prefer to use the latest versions instead, you can install them with:
    ```bash
    !wget http://setup.johnsnowlabs.com/colab.sh -O - | bash
    ```
  - Note: The latest versions may introduce breaking changes, so you might need to adjust the code accordingly.


In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 40.6 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `DeBertaForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DeBertaForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [10]:
from sparknlp.annotator import DeBertaForZeroShotClassification

zero_shot_classifier = DeBertaForZeroShotClassification.loadSavedModel(
      ONNX_MODEL,
      spark
      )\
    .setInputCols(["document", "token"]) \
    .setOutputCol("class") \
    .setCandidateLabels(["urgent", "mobile", "travel", "movie", "music", "sport", "weather", "technology"])

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [12]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your DebertaV2ForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [13]:
! ls -l {MODEL_NAME}_spark_nlp

ls: cannot access 'MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli_spark_nlp': No such file or directory


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny DeBertaForZeroShotClassification model 😊

In [14]:
zero_shot_classifier_loaded = DeBertaForZeroShotClassification.load("./{}_spark_nlp".format(ONNX_MODEL))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [15]:
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols("document") \
    .setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

text = [
    ["I have a problem with my iphone that needs to be resolved asap!!"],
    ["Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."],
    ["I have a phone and I love it!"],
    ["I really want to visit Germany and I am planning to go there next year."],
    ["Let's watch some movies tonight! I am in the mood for a horror movie."],
    ["Have you watched the match yesterday? It was a great game!"],
    ["We need to hurry up and get to the airport. We are going to miss our flight!"]
]

inputDataset = spark.createDataFrame(text, ["text"])

model = pipeline.fit(inputDataset)
result = model.transform(inputDataset)

result.select("text", "class.result").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------+---------+
|text                                                                                                          |result   |
+--------------------------------------------------------------------------------------------------------------+---------+
|I have a problem with my iphone that needs to be resolved asap!!                                              |[music]  |
|Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app.|[weather]|
|I have a phone and I love it!                                                                                 |[sport]  |
|I really want to visit Germany and I am planning to go there next year.                                       |[sport]  |
|Let's watch some movies tonight! I am in the mood for a horror movie.                                         |[music]  |
|Have you watche

That's it! You can now go wild and use hundreds of `DebertaV2ForSequenceClassification` models as zero-shot classifiers from HuggingFace 🤗 in Spark NLP 🚀